In [22]:
#!pip install python-binance
#!pip install matplotlib --upgrade
#!pip install catboost
#!pip install --upgrade mplfinance  
#!pip install ta
#!pip install sqlalchemy

In [23]:
import numpy as np
import pandas as pd

import time
import dateparser
import pytz
import json
import math  

import datetime as dt
from datetime import datetime, timedelta

from threading import Thread

from binance.exceptions import BinanceAPIException

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.pyplot import figure

#from mpl_finance import candlestick_ochl

#from sklearn import preprocessing,model_selection,neighbors,svm
#from sklearn.preprocessing import StandardScaler, RobustScaler, MaxAbsScaler
#from sklearn.metrics import accuracy_score, mean_squared_error, mean_absolute_error

#from catboost import CatBoostClassifier, CatBoostRegressor, Pool, cv

from tqdm import tqdm as tqdm

from ta import add_all_ta_features

from binance.client import Client
from binance import BinanceSocketManager

In [24]:
api_id = "y8F6zJDWGNXQu7ON2BLR8WX0UVidn4aNUkVxFb54RrIsBxCUvsbl3NtilPD0ECC5"
api_key = "F6RlQvAiFevAmurRaNCoY0XfXWBAm8xu4DHBWPyUwgFBYuqCK7iTd4AyEDn9l37l"
#api_id = ""
#api_key = ""
client = Client(api_id,api_key)
    
bsm =BinanceSocketManager(client)
    
socket = bsm.trade_socket("BTCUSDT")



In [25]:
print(datetime.now())

2021-08-28 15:24:23.101757


# Indikatörler

In [26]:
def exponential_moving_average(df, n):
  EMA = pd.Series(df['Close'].ewm(span=n, min_periods=n).mean(), name="EMA_" + str(n))
  df = df.join(EMA)
  return df

In [27]:
def momentum(df, n):
  M = pd.Series(df['Close'].diff(n), name="Momentum_"+str(n))
  df = df.join(M)
  return df

In [28]:
def bollinger_bands(df, n):
  MA = pd.Series(df['Close'].rolling(n, min_periods=n).mean())
  MSD = pd.Series(df['Close'].rolling(n, min_periods=n).std())
  b1 = 4 * MSD / MA
  B1 = pd.Series(b1, name = "BollingerB_" + str(n))
  df = df.join(B1)
  b2 = (df['Close'] - MA  + 2 * MSD) / (4 * MSD)
  B2 = pd.Series(b2, name='Bollinger%b_'+str(n))
  df = df.join(B2)
  return df

In [29]:
def stochastic_oscillator(df ,n):
  S0K = pd.Series((df['Close']- df['Low']) / (df['High']-df['Low']), name='S0k_' + str(n))
  df = df.join(S0K)
  return df

In [30]:
def relative_strength_index(df, n):
  i = 0
  UpI = [0]
  DoI = [0]

  while i + 1 <= df.index[-1]:
    UpMove = df.loc[i+1, 'High'] - df.loc[i, 'High']
    DoMove = df.loc[i+1, 'Low'] - df.loc[i, 'Low']
    if UpMove > DoMove and UpMove > 0:
      UpD = UpMove
    else:
      UpD = 0
    UpI.append(UpD)
    if DoMove > UpMove and DoMove > 0:
      DoD = DoMove
    else:
      DoD = 0
    DoI.append(DoD)
    i = i + 1

  UpI = pd.Series(UpI)
  DoI = pd.Series(DoI)
  PosDI = pd.Series(UpI.ewm(span=n, min_periods=n).mean())
  NegDI = pd.Series(DoI.ewm(span=n, min_periods=n).mean())
  RSI = pd.Series(PosDI / (PosDI + NegDI), name='RSI_' + str(n))
  df = df.join(RSI)
  return df

In [31]:
def common_channel_index(df, n):
  PP = (df['High'] + df['Low'] + df['Close']) / 3
  CCI = pd.Series((PP - PP.rolling(n, min_periods=n).mean()) / PP.rolling(n, min_periods=n).std(), name='CCI_' + str(n))
  df = df.join(CCI)
  return df

In [32]:
def standart_deviation(df, n):
  df = df.join(pd.Series(df['Close'].rolling(n, min_periods=n).std(), name='STD_' + str(n)))
  return df

# Binance Yardımcı Fonksiyonlar


In [33]:
def binanceDataFrame(klines):
  df = pd.DataFrame(klines.reshape(-1,12), dtype=float, columns=['Open Time',
                                                                 'Open',
                                                                 'High',
                                                                 'Low',
                                                                 'Close',
                                                                 'Volume',
                                                                 'Close Time',
                                                                 'Quote asset volume',
                                                                 'Number of trades',
                                                                 'Taker buy base asset volume',
                                                                 'Taker buy quote asset volume',
                                                                 'Can be ignored']);

  df['Open Time'] = pd.to_datetime(df['Open Time'], unit='ms')
  df['Close Time'] = pd.to_datetime(df['Close Time'], unit='ms')

  return df

In [34]:
def date_to_miliseconds(date_str):
  epoch = datetime.utcfromtimestamp(0).replace(tzinfo=pytz.utc)
  d = dateparser.parse(date_str)
  if d.tzinfo is None or d.tzinfo.utcoffset(d) is None:
    d = d.replace(tzinfo=pytz.utc)

  #return difference in time
  return int((d - epoch).total_seconds()*1000.0)

In [35]:
def interval_to_miliseconds(interval):
  ms = None
  seconds_per_unit = {
      "m": 60,
      "h": 60 * 60,
      "d": 24 * 60 * 60,
      "w": 7 * 24 * 60 * 60,
  }

  unit = interval[-1]
  if unit in seconds_per_unit:
    try:
      ms = int(interval[:-1]) * seconds_per_unit[unit] * 1000
    except:
      pass

  return ms

In [36]:
def get_historical_klines(symbol, interval, start_str, end_str=None):
  output_data=[]
  limit=500
  timeFrame = interval_to_miliseconds(interval)
  start_ts = date_to_miliseconds(start_str)
  end_ts = None
  if end_str:
    end_ts = date_to_miliseconds(end_str)
  
  idx = 0
  symbol_existed = False
  while True:
    #fetch
    temp_data = client.get_klines(symbol = symbol, 
                                  interval = interval, 
                                  limit = limit, 
                                  startTime = start_ts, 
                                  endTime = end_ts)
    
    if not symbol_existed and len(temp_data):
      symbol_existed = True

    if symbol_existed:
      output_data += temp_data

      start_ts = temp_data[len(temp_data) - 1][0] + timeFrame

    idx += 1

    if len(temp_data)<limit:
      break

    if idx % 3 == 0:
      time.sleep(1)

  return output_data

##  Sat-Satın al For Backtest

In [37]:
def satinal(bakiye,harcananpara,price):
  alinanmiktar = harcananpara/price
  kalanbakiye = bakiye - harcananpara
  return kalanbakiye,alinanmiktar

In [38]:
def sat(bakiye,satilanmiktar,price,malmiktar):
  eldeedilenpara = satilanmiktar*price
  mevcutbakiye = bakiye + eldeedilenpara
  yenimalmiktar = malmiktar - satilanmiktar
  return mevcutbakiye,yenimalmiktar

# 3 Ema Stratejisi ile Long Pozisyon 

In [39]:
## Binance'dan Verileri Alma ve Düzenleme

In [74]:

%%time
symbol = "BTCUSDT"
start = "23 Aug, 2021"
end = "29 Aug, 2021"
interval = Client.KLINE_INTERVAL_5MINUTE
klines = get_historical_klines(symbol,interval,start,end)
klines = np.array(klines)
df = binanceDataFrame(klines)


interval2 = Client.KLINE_INTERVAL_1HOUR
klines2 = get_historical_klines(symbol,interval2,start,end)
klines2 = np.array(klines2)
df2 = binanceDataFrame(klines2)


remove_cols = [c for c in df.columns if c not in ['Open Time','Close Time','Open','Close','High','Low','Volume']]
df.drop(remove_cols, axis=1, inplace=True)
df2.drop(remove_cols, axis=1, inplace=True)


df = exponential_moving_average(df,7)
df = exponential_moving_average(df,13)
df = exponential_moving_average(df,26)

df2 = exponential_moving_average(df2,7)
df2 = exponential_moving_average(df2,13)
df2 = exponential_moving_average(df2,26)


df2.drop(index=df2.index[:26], 
        axis=0, 
        inplace=True)

df.drop(index=df.index[:26], 
        axis=0, 
        inplace=True)

Wall time: 2.82 s


In [75]:
#1 saatlik grafikte grafıktekı mumların ardısık olarak ema_7'nın ustunde kapattıgı zaman dılımlerınde trade yapmak ıstıyoruz 
#o yuzden bu donemlerı tespıt edıyoruz
Trend = pd.Series(df2["Close"] - df2["EMA_7"] ,name = "Trend")
df2=df2.join(Trend)

test =df2[df2["Trend"] >10]
test = test[test["EMA_7"]>test["EMA_26"]]

timeintervallong=[]
for i in range (test.index.size - 2):
    if((test.iloc[i+1]["Open Time"] - test.iloc[i]["Open Time"]).seconds == 3600):
        timeintervallong.append((test.iloc[i]["Open Time"]))
        if((test.iloc[i+2]["Open Time"] - test.iloc[i+1]["Open Time"]).seconds != 3600):
            timeintervallong.append((test.iloc[i+1]["Open Time"]))
            
longtimeinterval=[]
longtimeinterval.append(timeintervallong[0])
for i in range (len(timeintervallong) - 2):
    if(timeintervallong[i+1].hour - timeintervallong[i].hour == 1 and timeintervallong[i+2].hour - timeintervallong[i+1].hour != 1):
        longtimeinterval.append((timeintervallong[i+1]))
        longtimeinterval.append((timeintervallong[i+2]))
longtimeinterval.append(timeintervallong[len(timeintervallong)-1])
print(len(longtimeinterval)) 

10


In [76]:
#5 DKlık grafiği 1 Saatlikte buldugumuz aralıkları kullanarak parçalıyoruz
df.set_index("Open Time",inplace=True)

d=pd.to_datetime(longtimeinterval)
e=list(range(len(longtimeinterval)-1))
df["longpossible"]=pd.cut(df.index,bins=d,labels=e,include_lowest=True)
df.dropna(inplace=True)

In [77]:
#Bu Şekilde Parcalayınca tek sayılar bizim istemedıgımız aralıklar ciftler ise bizim kullanacagımız aralıkları saglıyor.
df["longpossible"]=df["longpossible"].astype(int)
print(type(df.iloc[1]["longpossible"]))
x =df["longpossible"] % 2 == 0
df=df[x]
df = df[df["Close"] > df["EMA_26"]]
df = df[df["EMA_7"]>df["EMA_13"]]  
df = df[df["EMA_13"]>df["EMA_26"]]
df

<class 'numpy.int32'>


,Open,High,Low,Close,Volume,Close Time,EMA_7,EMA_13,EMA_26,longpossible
Open Time,,,,,,,,,,
2021-08-25 16:00:00,48817.41,48853.74,48740.98,48787.08,371.028350,2021-08-25 16:04:59.999000064,48658.040717,48538.844008,48337.375601,0
2021-08-25 16:05:00,48787.08,48790.79,48675.46,48725.59,248.013755,2021-08-25 16:09:59.999000064,48674.928038,48565.522007,48366.132223,0
2021-08-25 16:10:00,48725.59,48725.59,48653.60,48692.91,124.698704,2021-08-25 16:14:59.999000064,48679.423528,48583.720292,48390.337985,0
2021-08-25 16:15:00,48692.91,48763.09,48681.03,48731.64,146.936331,2021-08-25 16:19:59.999000064,48692.477646,48604.851679,48415.619615,0
2021-08-25 16:20:00,48731.63,48786.56,48692.67,48716.92,149.923916,2021-08-25 16:24:59.999000064,48698.588235,48620.861439,48437.938162,0
...,...,...,...,...,...,...,...,...,...,...
2021-08-28 01:50:00,49166.29,49170.00,49110.00,49143.81,41.361040,2021-08-28 01:54:59.999000064,49127.839885,49122.683337,49096.156451,6
2021-08-28 01:55:00,49143.81,49167.45,49095.41,49141.51,45.067590,2021-08-28 01:59:59.999000064,49131.257414,49125.372860,49099.515973,6
2021-08-28 06:50:00,48960.71,48998.06,48937.67,48996.73,69.506370,2021-08-28 06:54:59.999000064,48954.262589,48932.927966,48928.213865,8


## Backtest Yapma

In [78]:
#Bu stratejiyi x tarıhten y tarıhine kadar uygulasaydık ne olurduyu test edıyoruz
bakiye = 10.0
malmiktar = 0.0
maldayiz = False
karsayısı = 0
zararsayısı = 0
trendsayısı = 0;
carpan = 0.008
karzararorani = 2
for i in range(1,df.index.size - 1):
  low1 = df.iloc[i]["Low"]
  high1 = df.iloc[i]["High"]
  open1 = df.iloc[i]["Open"]
  close1 = df.iloc[i]["Close"]
  ema7 = df.iloc[i]["EMA_7"]
  longpossible = df.iloc[i]["longpossible"]
  ema13 = df.iloc[i]["EMA_13"]
  ema26 = df.iloc[i]["EMA_26"]
  tarih = df.index[i] 
  mumunustkismi = low1 + 3*((high1 - low1) / 5)
  #çekic sekli olusmussanın if versiyonu
  if(low1 < ema7 and close1 > mumunustkismi and open1 > mumunustkismi and maldayiz==False):
    bakiye,malmiktar = satinal(bakiye,bakiye,close1)
    maldayiz = True
    zararkes = low1 - low1*carpan
    karal = close1 + karzararorani*(close1 - zararkes)
    print("alım yapıldı kar yeri",karal,"zarar yeri",zararkes,"cekic ıle yapıldı ",bakiye*malmiktar," tarih ",tarih)
    print(close1)
   #yutan boga olusumunun if versiyonu
  elif(df.iloc[i -1]["Close"] < df.iloc[i-1]["Open"] and close1 > open1 and low1 < ema7 and maldayiz == False and close1 > df.iloc[i-1]["Open"] and ((df.iloc[i-1]["Open"] - df.iloc[i-1]["Close"]) < (close1 - open1))):
    bakiye,malmiktar = satinal(bakiye,bakiye,close1)
    maldayiz = True
    zararkes = open1 - open1*carpan
    karal = close1 + karzararorani*(close1 - zararkes)
    print("alım yapıldı kar yeri",karal,"zarar yeri",zararkes,"yutan boga ıle yapıldı ",bakiye*malmiktar," tarih ",tarih)
    print(close1)
  if(df.iloc[i+1]["longpossible"] != longpossible and maldayiz == True):
    bakiye,malmiktar = sat(bakiye,malmiktar,close1,malmiktar)
    maldayiz = False
    print("ürün trend değiştiği için satıldı ",close1,"yeni bakiye",bakiye," tarih ",tarih)
    trendsayısı += 1
  if(maldayiz == True and high1 >= karal and low1 > zararkes):
    bakiye,malmiktar = sat(bakiye,malmiktar,karal,malmiktar)
    maldayiz = False
    print("ürün karla satıldı ",bakiye," tarih ",tarih)
    print(karal)
    karsayısı += 1
  elif(maldayiz == True and high1 < karal and low1 <= zararkes):
    bakiye,malmiktar = sat(bakiye,malmiktar,zararkes,malmiktar)
    maldayiz = False
    print("ürün zararla satıldı ",bakiye," tarih ",tarih)
    print(zararkes)
    zararsayısı += 1
if(bakiye == 0):
    print("bakiye",malmiktar*close1)
else:
    print("bakiye",bakiye)
print("karsayısı",karsayısı)
print("zararsayısı",zararsayısı)
print("trend ",trendsayısı)

alım yapıldı kar yeri 49588.18655999999 zarar yeri 48303.366720000005 yutan boga ıle yapıldı  0.0  tarih  2021-08-25 16:15:00
48731.64
ürün trend değiştiği için satıldı  49003.37 yeni bakiye 10.055760487436912  tarih  2021-08-25 22:35:00
alım yapıldı kar yeri 50187.37935999999 zarar yeri 48646.40032 yutan boga ıle yapıldı  0.0  tarih  2021-08-26 00:35:00
49160.06
ürün zararla satıldı  9.950690666241952  tarih  2021-08-26 10:35:00
48646.40032
alım yapıldı kar yeri 47941.43744000001 zarar yeri 46538.281279999996 cekic ıle yapıldı  0.0  tarih  2021-08-26 10:40:00
47006.0
ürün trend değiştiği için satıldı  47453.83 yeni bakiye 10.045491708684686  tarih  2021-08-27 11:55:00
alım yapıldı kar yeri 49158.91520000001 zarar yeri 47856.2624 yutan boga ıle yapıldı  0.0  tarih  2021-08-27 15:25:00
48290.48
ürün trend değiştiği için satıldı  49022.89 yeni bakiye 10.197849245457103  tarih  2021-08-27 23:00:00
alım yapıldı kar yeri 50190.67800000001 zarar yeri 48772.176 cekic ıle yapıldı  0.0  tarih  

# 3 Ema Stratejisi ile Short Pozisyon

%%time
symbol = "BTCUSDT"
start = "14 Apr, 2021"
end = "22 Jun, 2021"
interval = Client.KLINE_INTERVAL_5MINUTE
klines = get_historical_klines(symbol,interval,start,end)
klines = np.array(klines)
df = binanceDataFrame(klines)


interval2 = Client.KLINE_INTERVAL_1HOUR
klines2 = get_historical_klines(symbol,interval2,start,end)
klines2 = np.array(klines2)
df2 = binanceDataFrame(klines2)


remove_cols = [c for c in df.columns if c not in ['Open Time','Close Time','Open','Close','High','Low','Volume']]
df.drop(remove_cols, axis=1, inplace=True)
df2.drop(remove_cols, axis=1, inplace=True)


df = exponential_moving_average(df,7)
df = exponential_moving_average(df,13)
df = exponential_moving_average(df,26)

df2 = exponential_moving_average(df2,7)
df2 = exponential_moving_average(df2,13)
df2 = exponential_moving_average(df2,26)


df2.drop(index=df2.index[:26], 
        axis=0, 
        inplace=True)

df.drop(index=df.index[:26], 
        axis=0, 
        inplace=True)

Trend = pd.Series(df2["Close"] - df2["EMA_7"] ,name = "Trend")
df2=df2.join(Trend)

test =df2[df2["Trend"] < 10]
test =test[test["EMA_7"]<test["EMA_13"]]
test = test[test["EMA_13"]<test["EMA_26"]]


timeintervallong=[]
for i in range (test.index.size - 2):
    if((test.iloc[i+1]["Open Time"] - test.iloc[i]["Open Time"]).seconds == 3600):
        timeintervallong.append((test.iloc[i]["Open Time"]))
        if((test.iloc[i+2]["Open Time"] - test.iloc[i+1]["Open Time"]).seconds != 3600):
            timeintervallong.append((test.iloc[i+1]["Open Time"]))
            
longtimeinterval=[]
longtimeinterval.append(timeintervallong[0])
for i in range (len(timeintervallong) - 2):
    if(timeintervallong[i+1].hour - timeintervallong[i].hour == 1 and timeintervallong[i+2].hour - timeintervallong[i+1].hour != 1):
        longtimeinterval.append((timeintervallong[i+1]))
        longtimeinterval.append((timeintervallong[i+2]))
longtimeinterval.append(timeintervallong[len(timeintervallong)-1])
print(len(longtimeinterval)) 

df.set_index("Open Time",inplace=True)

d=pd.to_datetime(longtimeinterval)
e=list(range(len(longtimeinterval)-1))
df["longpossible"]=pd.cut(df.index,bins=d,labels=e,include_lowest=True)
df.dropna(inplace=True)

df["longpossible"]=df["longpossible"].astype(int)
print(type(df.iloc[1]["longpossible"]))
x =df["longpossible"] % 2 == 0
df=df[x]
df = df[df["Close"] < df["EMA_26"]]
df = df[df["EMA_7"]<df["EMA_13"]]  
df = df[df["EMA_13"]<df["EMA_26"]]
df

#Bu stratejiyi x tarıhten y tarıhine kadar uygulasaydık ne olurduyu test edıyoruz
#ÇARPAN
carpan = 0.004

bakiye = 0.0
malmiktar = 1
maldayiz = False
karsayısı = 0
zararsayısı = 0
trendsayısı = 0;
baslangicbakiyesi = df.iloc[1]["Close"] * 0.01
for i in range(1,df.index.size - 1):
  low1 = df.iloc[i]["Low"]
  high1 = df.iloc[i]["High"]
  open1 = df.iloc[i]["Open"]
  close1 = df.iloc[i]["Close"]
  ema7 = df.iloc[i]["EMA_7"]
  longpossible = df.iloc[i]["longpossible"]
  tarih = df.index[i] 
  mumunaltkismi = high1 - 3*((high1 - low1) / 5)
  #çekic sekli olusmussanın if versiyonu
  if(high1 > ema7 and close1 < mumunaltkismi and open1 < mumunaltkismi and maldayiz==False ):
    bakiye,malmiktar = sat(bakiye,malmiktar,close1,malmiktar)
    maldayiz = True
    zararkes = high1 + high1*carpan
    karal = close1 - 2*(zararkes - close1)
    print(close1,"den satıs yapıldı kar yeri",karal,"zarar yeri",zararkes,"cekic ıle yapıldı ",close1*malmiktar," tarih ",tarih)
    
    print("bakiye ",bakiye,"malmiktar ",malmiktar)
   #yutan boga olusumunun if versiyonu
  elif(maldayiz == False and high1 > ema7 and df.iloc[i-1]["Close"] > df.iloc[i-1]["Open"]  and close1 < open1 and df.iloc[i-1]["Close"]<open1 and (df.iloc[i-1]["Close"] - df.iloc[i-1]["Open"])):
    bakiye,malmiktar = sat(bakiye,malmiktar,close1,malmiktar)
    maldayiz = True
    zararkes = high1 + high1*carpan
    karal = close1 - 2*(zararkes - close1)
    print(close1,"den satıs yapıldı kar yeri",karal,"zarar yeri",zararkes,"yutan boga ıle yapıldı ",close1*malmiktar," tarih ",tarih)
    
    print("bakiye ",bakiye,"malmiktar ",malmiktar)
  if(df.iloc[i+1]["longpossible"] != longpossible and maldayiz == True):
    bakiye,malmiktar = satinal(bakiye,bakiye,close1)
    maldayiz = False
    print("ürün trend değiştiği için alındı ",close1,"yeni bakiye",bakiye," tarih ",tarih)
    
    print("bakiye ",bakiye,"malmiktar ",malmiktar)
    trendsayısı += 1
  if(maldayiz == True and high1 < zararkes and low1 <= karal):
    bakiye,malmiktar = satinal(bakiye,bakiye,karal)
    maldayiz = False
    print(karal,"den ürün karla alındı ",bakiye," tarih ",tarih)
    
    print("bakiye ",bakiye,"malmiktar ",malmiktar)
    karsayısı += 1
  elif(maldayiz == True and high1 >= zararkes and low1 > karal):
    bakiye,malmiktar = satinal(bakiye,bakiye,zararkes)
    maldayiz = False
    print(zararkes,"den ürün zararla alındı ",bakiye," tarih ",tarih)
    
    print("bakiye ",bakiye,"malmiktar ",malmiktar)
    zararsayısı += 1

print("baslangıc bakiyesi 1 btc")
print("bakiye",bakiye/df.iloc[df.index.size - 1]["Close"]," btc")
print("malmiktarı ",malmiktar)
print("karsayısı",karsayısı)
print("zararsayısı",zararsayısı)
print("trend ",trendsayısı)
#SONUCLAR BURADA GOZULUYOR

# 3 Ema İle Güncel Olarak Al-Sat Yapma


maldayiz = False
maldayiz_short = False
carpan = 0.004
trendsayisi = 0
karsayisi = 0
zararsayisi = 0
bakiye = 0.0
malmiktar = 0.0
qty = 10
time_change = dt.timedelta(hours = 30)

while(True):
    
    time.sleep(2)
    new_time = datetime.now()-time_change
    new_time_str = new_time.strftime("%d %b, %Y")
    
    try:
        symbol = "BTCUSDT"
        start = new_time_str
        interval = Client.KLINE_INTERVAL_5MINUTE
        klines = get_historical_klines(symbol,interval,start)
        klines = np.array(klines)
        five_min_df = binanceDataFrame(klines)
    except BinanceAPIException as e:
        print(e,"5 dklık hata vermisti ama ben 60 saniye bekleyıp tekrar denıyecegım")
        time.sleep(60)
        symbol = "BTCUSDT"
        start = new_time_str
        interval = Client.KLINE_INTERVAL_5MINUTE
        klines = get_historical_klines(symbol,interval,start)
        klines = np.array(klines)
        five_min_df = binanceDataFrame(klines)
        
    remove_cols = [c for c in five_min_df.columns if c not in ['Open Time','Close Time','Open','Close','High','Low','Volume']]
        
    five_min_df.drop(remove_cols, axis=1, inplace=True)
    
    five_min_df = exponential_moving_average(five_min_df,7)
    five_min_df = exponential_moving_average(five_min_df,13)
    five_min_df = exponential_moving_average(five_min_df,26)
        
    five_min_df.drop(index=five_min_df.index[:26], 
            axis=0, 
            inplace=True)
        
    try:       
        symbol = "BTCUSDT"
        start = new_time_str
        interval2 = Client.KLINE_INTERVAL_1HOUR
        klines2 = get_historical_klines(symbol,interval2,start)
        klines2 = np.array(klines2)
        one_hour_df = binanceDataFrame(klines2)
        
    except BinanceAPIException as e:
        print(e,"1 saatlik hata vermisti ama ben 60 saniye bekleyıp tekrar denıyecegım")
        time.sleep(60)
        symbol = "BTCUSDT"
        start = new_time_str
        interval2 = Client.KLINE_INTERVAL_1HOUR
        klines2 = get_historical_klines(symbol,interval2,start)
        klines2 = np.array(klines2)
        one_hour_df = binanceDataFrame(klines2)
        
    remove_cols = [c for c in one_hour_df.columns if c not in ['Open Time','Close Time','Open','Close','High','Low','Volume']]
        
    one_hour_df.drop(remove_cols, axis=1, inplace=True)

    one_hour_df = exponential_moving_average(one_hour_df,7)
    one_hour_df = exponential_moving_average(one_hour_df,13)
    one_hour_df = exponential_moving_average(one_hour_df,26)


    one_hour_df.drop(index=one_hour_df.index[:26], 
            axis=0, 
            inplace=True)

    
    
 #////////////////////////////////////////////////////////////////////////////////////////////   
    mum_kapanıs_kontrol = one_hour_df.iloc[one_hour_df.index.size - 2]["Close"] - one_hour_df.iloc[one_hour_df.index.size- 2]["EMA_7"]
    ema_kontrol_1 = one_hour_df.iloc[one_hour_df.index.size- 2]["EMA_7"] - one_hour_df.iloc[one_hour_df.index.size- 2]["EMA_26"]
 
    
    mum_kapanis_kontrol2 = five_min_df.iloc[five_min_df.index.size - 2]["Close"] - five_min_df.iloc[five_min_df.index.size- 2]["EMA_26"]
    ema_kontrol_5min_1 = five_min_df.iloc[five_min_df.index.size - 2]["EMA_7"] - five_min_df.iloc[five_min_df.index.size- 2]["EMA_13"]
    ema_kontrol_5min_2 = five_min_df.iloc[five_min_df.index.size - 2]["EMA_13"] - five_min_df.iloc[five_min_df.index.size- 2]["EMA_26"]
    
    
    trend_kontrol = False
    trend_kontrol_5min = False
    
    trend_kontrol_short = False
    trend_kontrol_short_5min = False
    
    if(mum_kapanıs_kontrol > 0 and ema_kontrol_1 > 0 ):
        trend_kontrol = True
    if(mum_kapanis_kontrol2 > 0 and ema_kontrol_5min_1 > 0 and ema_kontrol_5min_2 > 0):
        trend_kontrol_5min = True
    if(mum_kapanıs_kontrol < 0 and ema_kontrol_1 < 0 ):
        trend_kontrol_short = True
    if(mum_kapanis_kontrol2 < 0 and ema_kontrol_5min_1 < 0 and ema_kontrol_5min_2 < 0):
        trend_kontrol_short_5min = True
        
        
    son_eleman_index = five_min_df.index.size + 25
    low1 = five_min_df.iloc[five_min_df.index.size - 2]["Low"]
    high1 = five_min_df.iloc[five_min_df.index.size - 2]["High"]
    open1 = five_min_df.iloc[five_min_df.index.size - 2]["Open"]
    close1 = five_min_df.iloc[five_min_df.index.size - 2]["Close"]
    ema7 = five_min_df.iloc[five_min_df.index.size - 2]["EMA_7"]
    tarih = five_min_df.iloc[five_min_df.index.size - 2]
    mumunustkismi = low1 + 3*((high1 - low1) / 5)
    mumunaltkismi = high1 - 3*((high1 - low1) / 5)
    
    
    
    balance = client.get_asset_balance(asset='USDT')
    orders = client.get_open_orders(symbol='BTCUSDT')
 #///////////////////////////////////////////////////////////////////////////////////////////// 
  #çekic sekli olusmussanın if versiyonu
    if(trend_kontrol == True and trend_kontrol_5min == True and low1 < ema7 and close1 > mumunustkismi and open1 > mumunustkismi and maldayiz==False and trend_kontrol == True):
        #////////////////////////////
        order = client.order_market_buy(
            symbol='BTCUSDT',
            quantity=0.0004)
        #////////////////////////////
        print(order)
        maldayiz = True
        zararkes = low1 - low1*carpan
        karal = close1 + 2*(close1 - zararkes)
        print("alim yeri ",close1," kar yeri",karal," zarar yeri ",zararkes," cekic ıle yapıldı ",bakiye*malmiktar," tarih ",tarih)
   #yutan boga olusumunun if versiyonu
    elif(trend_kontrol == True and trend_kontrol_5min == True and low1 < ema7 and maldayiz == False and five_min_df.iloc[five_min_df.index.size - 3]["Open"] > five_min_df.iloc[five_min_df.index.size - 3]["Close"]  and open1 < close1 and five_min_df.iloc[five_min_df.index.size - 3]["Open"]<close1 and (five_min_df.iloc[five_min_df.index.size - 3]["Open"] - five_min_df.iloc[five_min_df.index.size - 3]["Close"]) < (close1 - open1)):
        #////////////////////////////
        order = client.order_market_buy(
            symbol='BTCUSDT',
            quantity=0.0004)
        #////////////////////////////
        print(order)
        maldayiz = True
        zararkes = open1 - open1*carpan
        karal = close1 + 2*(close1 - zararkes)
        print("alım yeri ",close1," kar yeri ",karal," zarar yeri ",zararkes," yutan boga ıle yapıldı ",bakiye*malmiktar," tarih ",tarih)
    if(trend_kontrol == False and maldayiz == True):
        #////////////////////////////
        order = client.order_market_sell(
            symbol='BTCUSDT',
            quantity=0.0004)
        #////////////////////////////
        print(order)
        maldayiz = False
        print("ürün trend değiştiği için satıldı ", close1 ," yeni bakiye ", bakiye ," tarih ",tarih)
        trendsayisi += 1
    elif(maldayiz == True and five_min_df.iloc[five_min_df.index.size - 1]["High"] >= karal and five_min_df.iloc[five_min_df.index.size - 1]["Low"] > zararkes):
        #////////////////////////////
        order = client.order_market_sell(
            symbol='BTCUSDT',
            quantity=0.0004)
        #////////////////////////////
        print(order)
        maldayiz = False
        print("ürün karla ",karal ," fiyattan satıldı ",bakiye," tarih ",tarih)
        karsayisi += 1
    elif(maldayiz == True and five_min_df.iloc[five_min_df.index.size - 1]["High"] < karal and five_min_df.iloc[five_min_df.index.size - 1]["Low"] <= zararkes):
        #////////////////////////////
        order = client.order_market_sell(
            symbol='BTCUSDT',
            quantity=0.0004)
        #////////////////////////////
        print(order)
        maldayiz = False
        print("ürün zararla ",zararkes," fiyattan satıldı ",bakiye," tarih ",tarih)
        
        
        #/////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
        #/////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
        #/////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
        #/////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
        #/////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
        #/////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
        #/////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
        #/////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
    if(high1 > ema7 and close1 < mumunaltkismi and open1 < mumunaltkismi and maldayiz_short==False and trend_kontrol_short == True and trend_kontrol_short_5min == True ):
        order = client.order_market_sell(
            symbol='BTCUSDT',
            quantity=0.0004)
        #////////////////////////////
        print(order)
        maldayiz_short = True
        zararkes_short = high1 + high1*carpan
        karal_short = close1 - 2*(zararkes_short - close1)
        print("satis yeri ",close1," kar yeri",karal_short," zarar yeri ",zararkes_short," cekic ıle yapıldı ",close1*malmiktar," tarih ",tarih)
    #/////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
    elif(trend_kontrol_short == True and trend_kontrol_short_5min == True and high1 > ema7 and maldayiz_short == False and five_min_df.iloc[five_min_df.index.size - 3]["Close"] > five_min_df.iloc[five_min_df.index.size - 3]["Open"]  and close1 < open1 and five_min_df.iloc[five_min_df.index.size - 3]["Close"]<open1 and (five_min_df.iloc[five_min_df.index.size - 3]["Close"] - five_min_df.iloc[five_min_df.index.size - 3]["Open"]) < (open1- close1)):
        order = client.order_market_sell(
            symbol='BTCUSDT',
            quantity=0.0004)
        #////////////////////////////
        print(order)
        maldayiz_short = True
        zararkes_short = high1 + high1*carpan
        karal_short = close1 - 2*(zararkes_short - close1)
        print("satis yeri ",close1," kar yeri",karal_short," zarar yeri ",zararkes_short," yutan boga ıle yapıldı ",close1*malmiktar," tarih ",tarih)
    #////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
    elif(trend_kontrol_short == False and maldayiz_short == True):
        order = client.order_market_buy(
            symbol='BTCUSDT',
            quantity=0.0004)
        print(order)
        maldayiz_short = False
        print("ürün trend değiştiği için alındı ",close1,"yeni bakiye",bakiye," tarih ",tarih)
    #//////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
    elif(maldayiz_short == True and high1 < zararkes_short and low1 <= karal_short):
        order = client.order_market_buy(
            symbol='BTCUSDT',
            quantity=0.0004)
        print(order)
        maldayiz_short = False
        print(karal_short,"den ürün karla alındı ",bakiye," tarih ",tarih)
    #////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
    elif(maldayiz_short == True and high1 >= zararkes_short and low1 > karal_short):
        order = client.order_market_buy(
            symbol='BTCUSDT',
            quantity=0.0004)
        print(order)
        maldayiz_short = False
        print(karal_short,"den ürün zararla alındı ",bakiye," tarih ",tarih)
        